In [ ]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = read.delim("https://raw.githubusercontent.com/AdiPersonalWorks/Random/master/student_scores%20-%20student_scores.csv",stringsAsFactors=F, header = T, sep=",")

In [ ]:
head(data)

In [ ]:
library(Hmisc)

In [ ]:
describe(data)

In [ ]:
# list rows of data that have missing values
data[!complete.cases(data),]

In [ ]:
boxplot(data$Hours)

In [ ]:
library("dplyr")
library("ggpubr")

In [ ]:
#Density plot and Q-Q plot can be used to check normality visually
library("ggpubr")
ggdensity(data$Hours, 
          main = "Density plot of No. of study hours",
          xlab = "Study Hours")

In [ ]:
library("car")
qqPlot(data$Hours)

In [ ]:
#The R function shapiro.test() can be used to perform the Shapiro-Wilk test of normality for one variable (univariate)
shapiro.test(data$Hours)

In [ ]:
hist(data$Hours)

In [ ]:
lm = lm(Scores~Hours, data = data) #Create the linear regression
summary(lm) #Review the results

In [ ]:
plot(data, pch = 16, col = "blue") #Plot the results
abline(lm) #Add a regression line

In [ ]:
plot(lm$residuals, pch = 16, col = "red")

In [ ]:
plot(Scores~Hours, data = data)
abline(lm)

In [ ]:
library(openintro)
library(dplyr)
library(ggplot2)
library(e1071)

In [ ]:
skewness(data$Hours)

In [ ]:
lm_log.model = lm(log1p(Scores) ~ log1p(Hours), data = data)

In [ ]:
summary(lm_log.model)

In [ ]:
data$log_hours = log(data$Hours)

In [ ]:
hist(data$log_hours)

In [ ]:
ggdensity(data$log_hours, 
          main = "Density plot of log transformed hours",
          xlab = "Study Hours")

In [ ]:
lm_log.model2 = lm(log1p(Scores) ~ log_hours, data = data)

In [ ]:
summary(lm_log.model2)

In [ ]:
#Now we can drop the newly log transformed column as it has no signifanct effect
data <- subset(data,,-c(3))
head(data)

In [ ]:
#Create data partition using 80-20 rule
library(caret)
set.seed(99)
index<-createDataPartition(data$Scores, p=0.60, list=FALSE)
train<-data[index,]
test<-data[-index,]

In [ ]:
#Applying linear regression using lm()
set.seed(99)
caret_lm<-lm(Scores~.,train)#nzv,range,YeoJohnson
caret_lm_tst_pred<- predict(caret_lm, test)
plot(caret_lm_tst_pred, test$Scores,
    xlab= "Predicted", ylab= "Actual",
    main="Predicted vs Actual: Linear", col="blue", pch=18)
grid()
abline(0, 1, col = "red", lwd = 2)

In [ ]:
summary(caret_lm)

In [ ]:
postResample(caret_lm_tst_pred, test$Scores)

In [ ]:
#Using caret's train() function
set.seed(999)
control<-trainControl(method="repeatedcv", number=10, repeats=3)
caret_lm2<-train(Scores~.,train, method="lm", trControl=control, preprocess=c("center","scale","BoxCox"))#nzv,range,YeoJohnson
clv_lm_tst_pred2<- predict(caret_lm2, test)
plot(clv_lm_tst_pred2, test$Scores,
    xlab= "Predicted", ylab= "Actual",
    main="Predicted vs Actual: Linear", col="blue", pch=18)
grid()
abline(0, 1, col = "red", lwd = 2)

In [ ]:
summary(caret_lm2)

In [ ]:
postResample(clv_lm_tst_pred2, test$Scores)

In [ ]:
#Implementing Random Forest where I set mtry=no.of predictors/3
library(randomForest)
set.seed(999)
rf<-randomForest(Scores~.,train,mtry=0.33,importance=TRUE,ntrees=100)
rf
rf_tst_pred<- predict(rf, test)
plot(rf_tst_pred, test$Scores,
    xlab= "Predicted", ylab= "Actual",
    main="Predicted vs Actual: Linear", col="blue", pch=18)
grid()
abline(0, 1, col = "red", lwd = 2)

summary(rf)
#(lm_tst_rmse2 = calc_rmse(clv_lm_tst_pred2, test$CLV))#14.4812044544613
#summary(clv_lm2)$r.squared
postResample(rf_tst_pred, test$Scores)

In [ ]:
summary(lm)

In [ ]:
summary(lm_log.model)

*Finally, we can conclude simple linear regression gives me the better result than Random Forest. And, Residual standard error significantly dropped in case of log transformed model output.* 
**Scores = a + Hours*b**
You can see the values of the intercept and the slope for the scores. These “a” and “b” values plot a line between all the points of the data. So in this case, if there is a student, who studies for 5 hours per day, intercept(a) is 2.4837 and slope(b) is 9.7758, the model predicts (on average) that student socres around 2.4837 + (5 * 9.7758) = 51.33627